In [6]:
import pandas as pd

In [7]:
data = pd.read_excel('../raw_data/41592_2018_45_MOESM4_ESM.xlsx', skiprows=6)
genes = data['Standard Name'].dropna()

/home/vaslem/code/bioinformatics_ms/modern_data_analytics/.venv/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [15]:
readme = """ 1) DB						- database contributing the file (always "SGD" for this file)
 2) DB_Object_ID				- SGDID
 3) DB_Object_Symbol				- see below
 4) NOT 			(optional)	- 'NOT', 'contributes_to', or 'colocalizes_with' qualifier for a GO annotation, when needed
 5) GO ID					- unique numeric identifier for the GO term
 6) DB:Reference(|DB:Reference)			- the reference associated with the GO annotation
 7) Evidence					- the evidence code for the GO annotation
 8) With (or) From 		(optional)	- any With or From qualifier for the GO annotation
 9) Aspect					- which ontology the GO term belongs in
10) DB_Object_Name(|Name) 	(optional)	- a name for the gene product in words, e.g. 'acid phosphatase'
11) DB_Object_Synonym(|Synonym) (optional)	- see below
12) DB_Object_Type				- type of object annotated, e.g. gene, protein, etc.
13) taxon(|taxon)				- taxonomic identifier of species encoding gene product
14) Date					- date GO annotation was made
15) Assigned_by					- source of the annotation (e.g. SGD, UniProtKB, YeastFunc, bioPIXIE_MEFIT)"""
columns = [x[x.find(')') + 1:x.find('\t')].strip() for x in readme.split('\n')]

In [18]:
len(columns)

15

In [24]:
data = pd.read_csv('../raw_data/gene_association.sgd.20211018.gaf', sep='\t', skiprows=7,header=None).iloc[:, :len(columns)]

In [25]:
data.columns = columns

In [27]:
data.head()

,DB,DB_Object_ID,DB_Object_Symbol,NOT,GO ID,DB:Reference(|DB:Reference),Evidence,With (or) From,Aspect,DB_Object_Name(|Name),DB_Object_Synonym(|Synonym) (optional),DB_Object_Type,taxon(|taxon),Date,Assigned_by
0,SGD,S000001503,SPT23,enables,GO:0003674,GO_REF:0000015,ND,NaN,F,ER membrane protein involved in regulation of ...,YKL020C,protein,taxon:559292,20181102,SGD
1,SGD,S000000735,GCN4,involved_in,GO:1990139,PMID:31211995,IMP,NaN,P,bZIP transcriptional activator of amino acid b...,YEL009C|AAS3|ARG9|AAS101|amino acid starvation...,protein,taxon:559292,20200206,SGD
2,SGD,S000003026,RAD6,contributes_to,GO:0017116,PMID:9287349,IDA,NaN,F,Ubiquitin-conjugating enzyme (E2),YGL058W|PSO8|UBC2|E2 ubiquitin-conjugating pro...,protein,taxon:559292,20131012,SGD
3,SGD,S000001357,PRK1,involved_in,GO:0120133,PMID:9885245,IGI,SGD:S000001445,P,Ser/Thr protein kinase,YIL095W|PAK1|serine/threonine protein kinase PRK1,protein,taxon:559292,20181031,SGD
4,SGD,S000028746,FMP49,part_of,GO:0005739,PMID:14576278,HDA,NaN,C,Mitochondrial protein of unknown function,YER038W-A,protein,taxon:559292,20040923,SGD


In [44]:
go_terms_description = data[data['DB_Object_Symbol'].isin(genes)][['GO ID', 'DB_Object_Name(|Name)', 'Aspect']].drop_duplicates()
go_terms_description = go_terms_description.rename(columns={'GO ID': 'goId', 'DB_Object_Name(|Name)': 'goDescription', 'Aspect': 'family'})
go_terms_description.to_csv('../processed_data/goDescription.csv', index=False)


In [45]:
go_terms_description.head()

,goId,goDescription,family
0,GO:0003674,ER membrane protein involved in regulation of ...,F
2,GO:0017116,Ubiquitin-conjugating enzyme (E2),F
3,GO:0120133,Ser/Thr protein kinase,P
5,GO:0003724,ATP-dependent DEAD-box RNA helicase with stran...,F
6,GO:0015035,Gamma subunit of the oligosaccharyltransferase...,F


In [40]:
genesMapping = data[data['DB_Object_Symbol'].isin(genes)].groupby('DB_Object_Symbol')['GO ID'].agg(list).reset_index().rename(columns={'DB_Object_Symbol': 'gene', 'GO ID': 'goId'})
genesMapping.head()

,gene,goId
0,AAC1,"[GO:0005471, GO:0006783, GO:0006783, GO:000683..."
1,AAC3,"[GO:0005471, GO:0005471, GO:0009061, GO:000906..."
2,AAD10,"[GO:0047681, GO:0006081, GO:0005575, GO:004783..."
3,AAD14,"[GO:0047681, GO:0047681, GO:0006081, GO:000557..."
4,AAD15,"[GO:0047681, GO:0006081, GO:0005575, GO:004783..."


In [41]:
genesMapping.to_csv('../processed_data/genesMappingGO.csv', index=False)